In [189]:
import requests
import pandas as pd
import re 



url = 'https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt'
r = requests.get(url=url)

text = r.content.decode()[10454:]
text = re.sub(r'<<[^>]*>>', '', text)
text = text.split('\n')

In [190]:
#l = [' '.join(text[i:i+n]) for i in range(0,len(text),n)]


shake_text = [line for line in text if re.search(r'^\s{2}', line)]
for i in range(0,len(shake_text)):
    line = shake_text[i].strip('\n')
    if(re.search(r'^\s{2}[A-Z\s]{2,}\.', line)):
        index = line.find('.')
        shake_text[i] = line[index+1:]
        

shake_text = ' '.join(shake_text).split(' ')
shake_text = [word.lower() for word in shake_text if word != '']


In [191]:
data = []
data_labels = []
for i in range(0, int(len(shake_text)*.8),20):
    data.append(' '.join(shake_text[i:i+20]))
    data_labels.append(1)
print(len(data))

33483


In [192]:
non_shake_files = []
non_shake_files.append(open('data/non1', 'r').read().split(' '))
non_shake_files.append(open('data/non2', 'r').read().split(' '))
non_shake_files.append(open('data/non3', 'r').read().split(' '))
non_shake_files.append(open('data/non4', 'r').read().split(' '))
non_shake_files.append(open('data/non5', 'r').read().split(' '))
non_shake_files.append(open('data/non6', 'r').read().split(' '))
non_shake_files.append(open('data/non7', 'r').read().split(' '))
non_shake_files.append(open('data/non8', 'r').read().split(' '))
non_shake_files.append(open('data/non9', 'r').read().split(' '))
non_shake_files.append(open('data/non10', 'r').read().split(' '))
non_shake_files.append(open('data/non11', 'r').read().split(' '))
non_shake_files.append(open('data/non12', 'r').read().split(' '))
non_shake_files.append(open('data/non13', 'r').read().split(' '))
non_shake_files.append(open('data/non14', 'r').read().split(' '))
non_shake_files.append(open('data/non15', 'r').read().split(' '))

for file in non_shake_files:
    for i in range(0, len(file), 20):
        quote = ' '.join(file[i:i+20])
        new_quote = quote.replace('\n', ' ').lower()
        data.append(new_quote)
        data_labels.append(0)


In [193]:
print(len(data))

63542


In [194]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
  
vocab_size = 7500
tokenize = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size) 
tokenize.fit_on_texts(data)

import pickle
with open('tokenize.pickle', 'wb') as handle:
    pickle.dump(tokenize, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [195]:
from sklearn import model_selection

data = tokenize.texts_to_sequences(data)
x_train, x_test, y_train, y_test = model_selection.train_test_split(data, data_labels, test_size=0.3, random_state=42)

In [196]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=20)
x_test = keras.preprocessing.sequence.pad_sequences(x_test,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=20)

In [197]:
x_train.shape

(44479, 20)

In [198]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

In [199]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [200]:
x_val = x_train[10000:]
x_validation = x_train[:10000]

y_val = y_train[10000:]
y_validation = y_train[:10000]

print(len(x_val[0]))

20


In [201]:
history = model.fit(x_val,
                    y_val,
                    epochs=20,
                    batch_size=400,
                    validation_data=(x_validation, y_validation),
                    verbose=1)

Train on 34479 samples, validate on 10000 samples
Epoch 1/20
34479/34479 [==============================]34479/34479 [==============================] - 1s 34us/step - loss: 0.6822 - acc: 0.5557 - val_loss: 0.6615 - val_acc: 0.6850

Epoch 2/20
34479/34479 [==============================]34479/34479 [==============================] - 0s 10us/step - loss: 0.6144 - acc: 0.7644 - val_loss: 0.5517 - val_acc: 0.8137

Epoch 3/20
34479/34479 [==============================]34479/34479 [==============================] - 0s 10us/step - loss: 0.4687 - acc: 0.8400 - val_loss: 0.4018 - val_acc: 0.8520

Epoch 4/20
34479/34479 [==============================]34479/34479 [==============================] - 0s 10us/step - loss: 0.3432 - acc: 0.8749 - val_loss: 0.3150 - val_acc: 0.8760

Epoch 5/20
34479/34479 [==============================]34479/34479 [==============================] - 0s 10us/step - loss: 0.2736 - acc: 0.8974 - val_loss: 0.2690 - val_acc: 0.8922

Epoch 6/20
34479/34479 [================

In [202]:
results = model.evaluate(x_test, y_test)

print(results)

19063/19063 [==============================]19063/19063 [==============================] - 1s 34us/step

[0.17635273924499495, 0.93159523684624668]


In [219]:
quote = tokenize.texts_to_sequences(["I know you want this for life Taking pictures with all my ice"])
quote = keras.preprocessing.sequence.pad_sequences(quote, value=0,
                                                        padding='post',
                                                        maxlen=20)
quote.shape

print(quote)

[[   6   79    9  378   24   16  135  678 4201   14   30   13 2333    0
     0    0    0    0    0    0]]


In [220]:
print(model.predict(quote,verbose=1))

1/1 [==============================]1/1 [==============================] - 0s 16ms/step

[[ 0.02279586]]


In [205]:
model.save('shake_model.h5')

In [165]:
print(data[0])

[3033, 39, 2661, 2013, 44, 565, 2733, 9, 3222, 3780, 607, 171, 103, 292, 20, 18, 1, 95, 32]


In [206]:
with open('tokenize2.pickle', 'wb') as handle:
    pickle.dump(tokenize, handle, protocol=pickle.HIGHEST_PROTOCOL)